In [1]:
import numpy as np
import pandas as pd

import torch
from torch.utils.data import DataLoader
from chemprop.args import TrainArgs
from pytorch_lightning import seed_everything


import wandb
import deepchem as dc
import os
os.environ["WANDB_SILENT"] = "true"
wandb.login(key="27edf9c66b032c03f72d30e923276b93aa736429")


Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/mmasood1/.conda/envs/env_arslan/lib/python3.9/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


True

In [2]:
from utils.data_utils import scafoldsplit_train_test, dataloader_for_numpy
from utils.utils import wandb_init_model
from utils.models import Custom_Chemprop

from pytorch_lightning.callbacks import ModelCheckpoint
from utils.model_utils import pretrained_model
from utils.model_utils import get_chemprop_pred, compute_binary_classification_metrics_MT


In [5]:
TrainArgs.project_name = 'DMPNN_Tox21'
TrainArgs.model_name = 'Trial'

TrainArgs.target_file = "/projects/home/mmasood1/arslan_data_repository/Tox21/complete_Tox21.csv"
TrainArgs.input_dim = 1024
TrainArgs.train_frac = 0.8

TrainArgs.pretrained_dir = "/projects/home/mmasood1/Model_weights/invitro/Chemprop/fold_0/fold_0/model_0/"
TrainArgs.model_weights_dir = "/projects/home/mmasood1/Model_weights/preclinical_clinical/chemprop/"
TrainArgs.metadata_dir = '/projects/home/mmasood1/trained_model_predictions/Tox21/Chemprop/'
TrainArgs.pretrained_model = False

TrainArgs.depth = 3
TrainArgs.hidden_size = 300
TrainArgs.ffn_num_layers = 2
TrainArgs.ffn_hidden_size = 300
TrainArgs.num_of_tasks = None
TrainArgs.use_input_features = False
TrainArgs.dropout = 0.2
TrainArgs.batch_size = 50
TrainArgs.adding_bond_types = True
TrainArgs.atom_descriptors_size = 0

TrainArgs.scheduler_type = 'ReduceLROnPlateau'
TrainArgs.warmup_epochs = 2
TrainArgs.epochs = 20
TrainArgs.init_lr = 1e-4
TrainArgs.max_lr = 1e-3
TrainArgs.final_lr = 1e-4
TrainArgs.loss_function = "binary_cross_entropy"
TrainArgs.seed = 42

TrainArgs.accelerator = 'gpu'
TrainArgs.EarlyStopping = True
TrainArgs.return_trainer = True
TrainArgs.device = torch.device("cuda")
TrainArgs.compute_metrics_during_training = True
TrainArgs.num_forward_passes = 10
TrainArgs.n_query = 10
TrainArgs.sampling_strategy = "uniform"
TrainArgs.seed = 42
TrainArgs.compute_metric_after_n_epochs = 5

args = TrainArgs
args.dataset_type = 'classification'
args.metric = 'auc'
args.is_atom_bond_targets = False
args.use_target_weights = False
args.missing_label_representation = 'nan'


In [6]:
args = TrainArgs
args.dataset_type = 'classification'
args.metric = 'auc'
args.is_atom_bond_targets = False
args.use_target_weights = False
args.missing_label_representation = 'nan'


In [7]:
# get targets information
data = pd.read_csv(args.target_file)
target_names = data.loc[:, "NR-AR":"SR-p53"].columns.tolist()

args.num_of_tasks = len(target_names)
args.selected_tasks = target_names


In [8]:
np.random.seed(0)
# seed_everything(seed = config["seed"])
train_set, test_set = scafoldsplit_train_test(target_file=args.target_file,
                                              selected_tasks=args.selected_tasks,
                                              FP_size=args.input_dim,
                                              train_frac=args.train_frac,)

randomstratifiedsplitter = dc.splits.RandomStratifiedSplitter()
pool_set, val_set, initial_set = randomstratifiedsplitter.train_valid_test_split(train_set,
                                                                                 frac_train=0.85,
                                                                                 frac_valid=0.134,
                                                                                 frac_test=0.016,
                                                                                 seed=42)

print("train_set", sorted(np.nansum(train_set.y, axis=0)))
print("test_set", sorted(np.nansum(test_set.y, axis=0)))
print("pool_set", sorted(np.nansum(pool_set.y, axis=0)))
print("val_set", sorted(np.nansum(val_set.y, axis=0)))
print("initial_set", sorted(np.nansum(initial_set.y, axis=0)))
train_set.y.shape, test_set.y.shape, pool_set.y.shape, val_set.y.shape, initial_set.y.shape


train_test_features (6264, 1024) (1567, 1024)
train_test_targets (6264, 12) (1567, 12)
train_set [132.0, 193.0, 196.0, 208.0, 251.0, 276.0, 281.0, 300.0, 589.0, 648.0, 711.0, 718.0]
test_set [44.0, 50.0, 54.0, 58.0, 68.0, 91.0, 92.0, 145.0, 147.0, 179.0, 207.0, 224.0]
pool_set [109.0, 164.0, 164.0, 172.0, 219.0, 236.0, 236.0, 255.0, 499.0, 555.0, 602.0, 608.0]
val_set [18.0, 25.0, 25.0, 27.0, 31.0, 33.0, 38.0, 39.0, 81.0, 84.0, 94.0, 102.0]
initial_set [3.0, 4.0, 5.0, 5.0, 6.0, 7.0, 7.0, 9.0, 9.0, 9.0, 9.0, 14.0]


((6264, 12), (1567, 12), (5324, 12), (839, 12), (101, 12))

In [9]:
def convert_to_dataframe(dc_datset, tasks_name):
    dataset = pd.DataFrame(dc_datset.y)
    dataset.columns = tasks_name
    dataset.insert(0, "SMILES", dc_datset.ids)
    return dataset

def convert_dataframe_to_dataloader(dataframe, args, shuffle = False):
    dataloader = DataLoader(dataloader_for_numpy(dataframe.SMILES.values, dataframe[args.selected_tasks].values, x_type='SMILES'),
                              batch_size=args.batch_size,
                              pin_memory=False,
                              num_workers=4,
                              shuffle=shuffle,
                              persistent_workers=True)
    return dataloader


In [10]:
# Who cares about deepchem data_object, trash it
initial_set = convert_to_dataframe(initial_set, args.selected_tasks)
val_set = convert_to_dataframe(val_set, args.selected_tasks)
pool_set = convert_to_dataframe(pool_set, args.selected_tasks)
test_set = convert_to_dataframe(test_set, args.selected_tasks)


In [11]:
train_dataloader = convert_dataframe_to_dataloader(dataframe= initial_set, args = args, shuffle= True)
val_dataloader = convert_dataframe_to_dataloader(dataframe= val_set, args = args, shuffle= False)
test_dataloader = convert_dataframe_to_dataloader(dataframe= test_set, args = args, shuffle= False)
pool_dataloader = convert_dataframe_to_dataloader(dataframe= pool_set, args = args, shuffle= False)

In [12]:

args.steps_per_epoch = len(train_dataloader)
args.model_name = f'd{args.depth}_MPN_h{args.hidden_size}_ffn_h{args.ffn_hidden_size}_DO{args.dropout}'

trained_model, run, trainer = wandb_init_model(Custom_Chemprop,
                                               args,
                                               train_dataloader,
                                               val_dataloader,
                                               model_type='chemprop')
checkpoint_callback = [
    cb for cb in trainer.callbacks if isinstance(cb, ModelCheckpoint)][0]
metric_to_optimize = checkpoint_callback.best_model_score.item()

# model = Custom_Chemprop(args)
# checkpoint = torch.load(checkpoint_callback.best_model_path, map_location=lambda storage, loc: storage)
# model.load_state_dict(checkpoint['state_dict'])
# model.eval()
wandb.finish()


20


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/mmasood1/.conda/envs/env_arslan/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/home/mmasood1/.conda/envs/env_arslan/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:634: Checkpoint directory /projects/home/mmasood1/Model_weights/preclinical_clinical/chemprop/ exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type              | Params
----------------------------------------------
0 | loss_fn | BCEWithLogitsLoss | 0     
1 | encoder | MPN               | 264 K 
2 | readout | Sequential        | 93.9 K
----------------------------------------------
358 K     Trainabl

6 60


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/projects/home/mmasood1/TG GATE/active_learning/utils/models.py:214: RuntimeWarning: Mean of empty slice
  return np.nanmean(roc_score), np.nanmean(blc_acc), np.nanmean(sensitivity), np.nanmean(specificity), np.nanmean(AUPR), np.nanmean(f1_score), np.nanmean(average_precision)
/home/mmasood1/.conda/envs/env_arslan/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

/projects/home/mmasood1/TG GATE/active_learning/utils/models.py:214: RuntimeWarning: Mean of empty slice
  return np.nanmean(roc_score), np.nanmean(blc_acc), np.nanmean(sensitivity), np.nanmean(specificity), np.nanmean(AUPR), np.nanmean(f1_score), np.nanmean(average_precision)
/projects/home/mmasood1/TG GATE/active_learning/utils/models.py:214: RuntimeWarning: Mean of empty slice
  return np.nanmean(roc_score), np.nanmean(blc_acc), np.nanmean(sensitivity), np.nanmean(specificity), np.nanmean(AUPR), np.nanmean(f1_score), np.nanmean(average_precision)
`Trainer.fit` stopped: `max_epochs=20` reached.


In [17]:
from scipy.special import expit

def enable_dropout(model):
    """ Function to enable the dropout layers during test-time """
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            m.train()


def get_chemprop_pred(dataloader, model, n_samples, n_classes, cal_uncert=False, num_forward_passes=1):

    dropout_predictions = np.empty((0, n_samples, n_classes))
    device = torch.device('cuda')
    model = model.eval()
    model = model.to(device)

    if cal_uncert == True:
        enable_dropout(model)

    for i in range(num_forward_passes):
        preds, targets = [], []
        for batch in dataloader:
            smiles, batch_targets = batch
            smiles = [[SMILES] for SMILES in smiles]

            with torch.no_grad():
                batch_preds = model(smiles)
                batch_preds = batch_preds.tolist()
                preds.extend(batch_preds)
                targets.extend(batch_targets.cpu().detach().tolist())
        preds = expit(np.array(preds))
        targets = np.array(targets)

        dropout_predictions = np.vstack(
            (dropout_predictions, preds[np.newaxis, :, :]))
        pred_mean = np.mean(dropout_predictions, axis=0)
        pred_var = np.var(dropout_predictions, axis=0)

    return targets, pred_mean, pred_var, dropout_predictions


In [18]:
def BALD_acquisition_function(pred):
    '''
    pred: probability (repeats, mol, tasks)
    '''
    pred_mean = pred.mean(axis=0)
    epsilon = 1e-10
    # Calculating entropy across multiple MCD forward passes
    # input (mol, tasks) --> (mol, tasks)
    H = - pred_mean * np.log(pred_mean + epsilon)  # shape (n_samples,task)
    E_H = np.mean(-pred * np.log(pred + epsilon), axis=0)
    # Calculating mutual information across multiple MCD forward passes
    MI = H - E_H
    return MI


def get_top_indices(array_2d, topk):

    # Flatten the array and get indices of N largest values
    flat_indices = np.argpartition(array_2d.flatten(), -topk)[-topk:]

    # Sort the top N values and their corresponding indices
    sorted_indices = flat_indices[np.argsort(
        array_2d.flatten()[flat_indices])][::-1]

    # Convert flat indices to 2D indices
    indices_2d = np.unravel_index(sorted_indices, array_2d.shape)

    return indices_2d


In [19]:
def get_random_indices(pool_set, arg):
    query_mol = np.random.choice(range(pool_set.shape[0]), size= arg.n_query, replace=False)
    query_task = np.random.choice(range(len(args.selected_tasks)), size= arg.n_query, replace=True)
    return (query_mol, query_task)

In [20]:
# get model predictions on test set
model = trained_model.eval()
targets, pred_mean, pred_var, all_pred = get_chemprop_pred(test_dataloader, model,
                                                           n_samples=test_set.shape[0],
                                                           n_classes=len(args.selected_tasks),
                                                           cal_uncert=False,
                                                           num_forward_passes=1)

# compute test metrics
metrics = compute_binary_classification_metrics_MT(targets, pred_mean, missing='nan')
metrics = metrics.append(metrics.mean(), ignore_index=True)
metrics.insert(0, 'Tasks', args.selected_tasks + ['mean'])
# metrics.to_csv(result_dir + f'{model_type[i]}_{label}_metrics.csv', index=False)


/tmp/ipykernel_30780/3917675102.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics = metrics.append(metrics.mean(), ignore_index=True)


In [23]:
# get predictions on pool_set
args.n_query = 10
args.num_forward_passes = 3
if args.sampling_strategy == "BALD":
    targets, pred_mean, pred_var, all_pred = get_chemprop_pred(pool_dataloader, model, 
                                n_samples= pool_set.shape[0], 
                                    n_classes= len(args.selected_tasks),
                                    cal_uncert= True, 
                                    num_forward_passes = args.num_forward_passes)
    
    acquisition = BALD_acquisition_function(all_pred)
    # We should not query with missing labels, so hide it
    nan_mask = ~np.isnan(pool_set[args.selected_tasks].values)
    acquisition = acquisition * nan_mask
    print(np.max(acquisition))

    # Get location of TopGuns
    top_indices = get_top_indices(acquisition, args.n_query)
    acquisition[top_indices]

if args.sampling_strategy == "uniform":
    top_indices = get_random_indices(pool_set, args)

In [24]:
top_indices

(array([4354, 1821, 2067, 4636, 1448, 3146, 1377, 1293, 4975, 2492]),
 array([11,  3,  8,  7, 10,  7, 11,  2,  4,  3]))

In [25]:
def get_query_set(pool_set, top_indices):

    # get top guns
    selected_mol = pool_set.iloc[top_indices[0],0]

    # diffuse all others ~y_ik
    p_set = pool_set.iloc[:, 1:].copy(deep=True)
    mask = np.ones_like(p_set, dtype=bool)
    mask[top_indices] = False
    p_set[mask] = np.nan

    # pull our guys (y_ik)
    selected_tasks = p_set.iloc[top_indices[0], :]
    top_guns = pd.concat([selected_mol, selected_tasks], axis = 1).reset_index(drop = True)
    return top_guns

In [26]:
def update_training_set(train_set, query_set):

    # Divide and conquer
    # convert query compounds into two sets

    already_existing_compound = query_set[query_set.SMILES.isin(
        train_set.SMILES)]
    new_compound = query_set[~query_set.SMILES.isin(train_set.SMILES)]

    # merge with new compound
    train_set = pd.merge(train_set, new_compound,
                         on=train_set.columns.to_list(),
                         how="outer")

    # convert missing labels --> observed labels
    train_set.combine_first(already_existing_compound)
    return train_set


In [27]:
def remove_queried_index_from_pool_set(pool_set, top_indices):
    # update pool set
    SMILES = pool_set.iloc[:,0]
    updated_poolset = pool_set.iloc[:, 1:].copy(deep=True)
    mask = np.zeros_like(updated_poolset, dtype=bool)
    mask[top_indices] = True
    updated_poolset[mask] = np.nan
    updated_poolset = pd.concat([SMILES, updated_poolset], axis = 1).reset_index(drop = True)
    updated_poolset.dropna(axis = 0, how = 'all')
    return updated_poolset

In [28]:
query_set = get_query_set(pool_set, top_indices)
updated_training = update_training_set(initial_set, query_set)
updated_poolset = remove_queried_index_from_pool_set(pool_set, top_indices)

initial_counts = initial_set.iloc[:, 1:].count()
query_counts = query_set.iloc[:, 1:].count()
updated_training_counts = updated_training.iloc[:, 1:].count()

initial_poolset_counts = pool_set.iloc[:, 1:].count()
updated_poolset_counts = updated_poolset.iloc[:, 1:].count()


# Use an assertion to check if the counts are equal
assert updated_training_counts.equals(initial_counts + query_counts), "Training_count,Queryset counts are not equal"
assert initial_poolset_counts.equals(updated_poolset_counts + query_counts), "Poolset, Queryset count are not equal"


In [29]:
pool_set.shape, updated_poolset.shape

((5324, 13), (5324, 13))

In [225]:
p_set

,SMILES,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53
0,CC(O)(P(=O)(O)O)P(=O)(O)O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CC(C)(C)OOC(C)(C)CCC(C)(C)OOC(C)(C)C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
2,CCCCCCCC(=O)[O-].CCCCCCCC(=O)[O-].[Zn+2],NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
3,CC(C)COC(=O)C(C)C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,C=C(C)C(=O)OCCOC(=O)C(=C)C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5319,CC(C)(Oc1ccc(Cl)cc1)C(=O)OCCCOC(=O)C(C)(C)Oc1c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
5320,O=S1(=O)c2cccc3cccc(c23)N1CCCN1CCN(c2ccc(F)cc2...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,0.0,NaN,NaN
5321,Cc1oc(-c2ccccc2)nc1CCC(=O)c1ccc(CC2SC(=O)NC2=O...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN
5322,CC1CCCCN1CCCOC(=O)c1ccc(OC2CCCCC2)cc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN


In [39]:
for i, j in zip(all_pred[0, :, 0], all_pred[1, :, 0]):
    print(np.round(i, 3), np.around(j, 3), np.round(np.var([i, j]), 5))


0.061 0.06 0.0
0.043 0.087 0.00049
0.006 0.03 0.00014
0.042 0.071 0.00021
0.037 0.019 8e-05
0.069 0.059 3e-05
0.022 0.057 0.00031
0.014 0.012 0.0
0.016 0.033 8e-05
0.002 0.001 0.0
0.003 0.001 0.0
0.059 0.076 7e-05
0.039 0.008 0.00024
0.054 0.024 0.00021
0.067 0.085 8e-05
0.035 0.026 2e-05
0.225 0.171 0.00071
0.039 0.023 7e-05
0.049 0.013 0.00032
0.002 0.003 0.0
0.012 0.017 1e-05
0.043 0.048 1e-05
0.12 0.035 0.00178
0.945 0.93 6e-05
0.025 0.014 3e-05
0.053 0.12 0.00113
0.051 0.038 4e-05
0.071 0.075 0.0
0.033 0.058 0.00015
0.022 0.031 2e-05
0.041 0.009 0.00025
0.025 0.037 4e-05
0.041 0.031 3e-05
0.036 0.04 1e-05
0.07 0.053 7e-05
0.071 0.058 4e-05
0.039 0.042 0.0
0.043 0.035 2e-05
0.026 0.042 7e-05
0.013 0.011 0.0
0.03 0.019 3e-05
0.017 0.038 0.00011
0.072 0.02 0.0007
0.086 0.053 0.00027
0.039 0.033 1e-05
0.04 0.063 0.00013
0.015 0.03 6e-05
0.019 0.038 9e-05
0.01 0.017 1e-05
0.029 0.024 1e-05
0.055 0.046 2e-05
0.075 0.123 0.00057
0.039 0.058 9e-05
0.064 0.033 0.00024
0.037 0.029 2e-05
0.0

In [17]:
checkpoint_callback.best_model_path


'/projects/home/mmasood1/Model_weights/preclinical_clinical/chemprop/model-epoch=11-val_BCE_loss=0.19.ckpt'

In [14]:
models_chkpoints = [checkpoint_callback.best_model_path,
                    checkpoint_callback.last_model_path]
model_type = ["best_model", "last_model"]

data_dir = args.metadata_dir + "predicitons/"
result_dir = args.metadata_dir + "Results/"

for i, chk_path in enumerate(models_chkpoints):
    print(i)
    args.pretrained_dir = chk_path
    model = pretrained_model(Custom_Chemprop, args)
    model = model.eval()

    targets, preds, pred_var = get_chemprop_pred(val_dataloader, model,
                                                 n_samples=val_y.shape[0],
                                                 n_classes=val_y.shape[1],
                                                 cal_uncert=True,
                                                 num_forward_passes=args.num_forward_passes)
    metrics = compute_binary_classification_metrics_MT(
        targets, preds, missing='nan')
    print(model_type[i], metrics.mean())
    metrics = metrics.append(metrics.mean(), ignore_index=True)
    metrics.insert(0, 'Tasks', args.selected_tasks + ['mean'])

    obs = pd.DataFrame(targets)
    obs.columns = args.selected_tasks
    obs.insert(0, 'SMILES', test_set.ids)

    pred = pd.DataFrame(preds)
    pred.columns = args.selected_tasks
    pred.insert(0, 'SMILES', test_set.ids)

    pred_var = pd.DataFrame(pred_var)
    pred_var.columns = args.selected_tasks
    pred_var.insert(0, 'SMILES', test_set.ids)

    label = 'test'
    # obs.to_csv(data_dir + f'{model_type[i]}_{label}_obs.csv', index=False)
    # pred.to_csv(data_dir + f'{model_type[i]}_{label}_pred.csv', index=False)
    # pred_var.to_csv(data_dir + f'{model_type[i]}_{label}_pred.csv', index=False)
    # metrics.to_csv(result_dir + f'{model_type[i]}_{label}_metrics.csv', index=False)

print("script completed")


0
Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Moving model to cuda
best_model balanced_acc         0.544626
f1_score             0.202154
specificity          0.305844
sensitivity          0.783408
roc_auc              0.473830
AUPR                 0.097593
average_precision    0.100683
dtype: float64
1
Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained pa

/tmp/ipykernel_24192/3954520195.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics = metrics.append(metrics.mean(), ignore_index= True)


last_model balanced_acc         0.528686
f1_score             0.189478
specificity          0.446164
sensitivity          0.611208
roc_auc              0.432481
AUPR                 0.094822
average_precision    0.096823
dtype: float64
script completed


/tmp/ipykernel_24192/3954520195.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics = metrics.append(metrics.mean(), ignore_index= True)


pd.read_csv("/projects/home/mmasood1/trained_model_predictions/Tox21/Chemprop/Results/best_model_test_metrics.csv")

pd.read_csv("/projects/home/mmasood1/trained_model_predictions/Tox21/Chemprop/Results/last_model_test_metrics.csv")

args.num_forward_passes = 2
models_chkpoints = ["/projects/home/mmasood1/Model_weights/preclinical_clinical/chemprop/model-epoch=18-val_BCE_loss=0.19.ckpt"]
model_type = ["best_model"]

data_dir = args.metadata_dir + "predicitons/"
result_dir = args.metadata_dir + "Results/"

for i, chk_path in enumerate(models_chkpoints):
    print(i)
    args.pretrained_dir = chk_path
    trained_model = pretrained_model(Custom_Chemprop,args)
    trained_model = trained_model.eval()

    targets, preds, pred_var = get_chemprop_pred(val_dataloader, trained_model, 
                                   n_samples= val_y.shape[0], 
                                   n_classes= val_y.shape[1],
                                   cal_uncert= True, 
                                   num_forward_passes = args.num_forward_passes)
    metrics = compute_binary_classification_metrics_MT(targets, preds, missing = 'nan')
    metrics = metrics.append(metrics.mean(), ignore_index= True)
    metrics.insert(0, 'Tasks', args.selected_tasks + ['mean'])

print("script completed")

In [4]:
results = pd.read_csv(
    "/projects/home/mmasood1/trained_model_predictions/Tox21/Chemprop/Results/test_metrics.csv")
results.round(3)


,Tasks,balanced_acc,f1_score,specificity,sensitivity,roc_auc,AUPR,average_precision
0,NR-AR,0.727,0.568,0.989,0.466,0.752,0.485,0.486
1,NR-AR-LBD,0.789,0.565,0.920,0.659,0.771,0.430,0.443
2,NR-AhR,0.766,0.519,0.717,0.816,0.828,0.527,0.529
3,NR-Aromatase,0.687,0.365,0.852,0.522,0.743,0.324,0.331
4,NR-ER,0.642,0.378,0.906,0.379,0.684,0.373,0.374
5,NR-ER-LBD,0.699,0.362,0.917,0.480,0.744,0.274,0.286
6,NR-PPAR-gamma,0.704,0.275,0.685,0.722,0.750,0.175,0.181
7,SR-ARE,0.675,0.500,0.810,0.540,0.722,0.467,0.469
8,SR-ATAD5,0.679,0.277,0.697,0.662,0.729,0.194,0.200
9,SR-HSE,0.704,0.393,0.902,0.505,0.760,0.305,0.313


In [ ]:
results = pd.read_csv(
    "/projects/home/mmasood1/trained_model_predictions/Tox21/Chemprop/Results/test_metrics.csv")


In [18]:
pred_var = pd.read_csv(
    "/projects/home/mmasood1/trained_model_predictions/Tox21/Chemprop/predicitons/test_pred.csv")
pred_var.mean().mean().round(6)


/tmp/ipykernel_24431/379112739.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  pred_var.mean().mean().round(6)


0.001732

In [22]:
data = pd.read_csv(args.target_file)
data = data.loc[:, args.selected_tasks]

target_weights = (data == 0).sum() / (data == 1).sum()
args.target_weights = target_weights.values
# normalize target weights (Coming from Chemprop)
avg_weight = sum(args.target_weights)/len(args.target_weights)
self.target_weights = [w/avg_weight for w in args.target_weights]


,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53
0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
7827,1.0,1.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0
7828,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
7829,1.0,1.0,0.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
